In [ ]:
"""
Copyright 2022 Huawei Technologies Co., Ltd

CREATED:  2022-10-04 13:12:13
MODIFIED: 2022-12-20 06:48:45
"""

# -*- coding:utf-8 -*-
import cv2, sys
import numpy as np
import random as rnd
#
from PIL import Image
from src.model import preprocessing, postprocessing
from src.postprocessing import detect, array2detections, nms
# define acllite library path
sys.path.append('../../Common/acllite')
# modules of acllite library
from acllite_model import AclLiteModel
from acllite_resource import AclLiteResource

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# define device id & model path
device_id = 0
model_path = "model/yolov4.om"

# define image & coco_names paths
img_path = "data/person.jpg"
coco_names_path = "data/coco.names"

In [ ]:
# initialize acllite library
acl_resource = AclLiteResource()
acl_resource.init()
print('[Device] NPU Device id:',acl_resource.device_id)
# load *.om model
model = AclLiteModel(model_path)

In [ ]:
# read image
img_org_bgr = cv2.imread(img_path)

In [ ]:
data, model_input_height, model_input_width = preprocessing(img_org_bgr,model._model_desc)

%timeit -n 10 -r 3
result_list = model.execute([data,])

In [ ]:
# 3) post-processing the raw predictions
bboxes = postprocessing(result_list,img_org_bgr,coco_names_path,model._model_desc)

In [ ]:
# draw bounding boxes
img1 = ImageDraw.Draw(img)

for bbox in bboxes:
    color ="#FF"+''.join([rnd.choice('0123456789ABCDEF') for j in range(4)])
    img1.rectangle(list(bbox[:4]), outline =color, width=3)
    print(bbox[:4], bbox[4], bbox[5], coco_labels[int(bbox[5])])
for bbox in bboxes:
    img1.rectangle((bbox[0],bbox[1],bbox[0]+45,bbox[1]+15),outline = (255,255,255),width=10)
    img1.text((bbox[0]+5, bbox[1]),coco_labels[int(bbox[5])],(0,0,0),thickness=10)

In [ ]:
img